In [ ]:
!pip install -U langchain langchain-core langchain-community langchain-google-genai langchain-openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from google.colab import userdata # 코랩 Secrets 기능 사용을 위한 임포트
from langchain_google_genai import ChatGoogleGenerativeAI
# 코랩 Secrets에서 GOOGLE_API_KEY를 가져오기
try:
    google_api_key = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = google_api_key
    print("GOOGLE_API_KEY가 환경 변수로 설정되었습니다.")
except userdata.SecretNotFoundError:
    print("오류: 'GOOGLE_API_KEY' 비밀을 코랩 Secrets에 설정해주세요. ")
    exit() # API 키가 없으면 스크립트 종료
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")
print(f"Selected LLM: {llm.__class__.__name__}")

GOOGLE_API_KEY가 환경 변수로 설정되었습니다.
Selected LLM: ChatGoogleGenerativeAI


In [ ]:
#데이터베이스 저장된 대화 기록 확인
import sqlite3
import json # 'message' 컬럼의 JSON 문자열을 파싱하기 위함
def get_history_from_db(session_id):
  db_path = f"/content/drive/MyDrive/LangChain/{session_id}_chat.db"
  conn = sqlite3.connect(db_path)
  cursor = conn.cursor()
  print(f"데이터베이스 '{db_path}'에 성공적으로 연결되었습니다.")

  cursor.execute("SELECT * FROM message_store")
  print("\n대화 기록")
  rows = cursor.fetchall()
  for row in rows:
    msg_id, session_id, message_json = row
    try:
        # 'message' 컬럼은 JSON 문자열이므로 파싱
        message_data = json.loads(message_json)
        msg_type = message_data.get("type", "unknown")

        # LangChain 메시지의 실제 내용은 'data' 딕셔너리 안에 'content' 키로 저장
        msg_content = message_data.get("data", {}).get("content", "[내용 없음]")

        print(f"ID: {msg_id}, Session: {session_id}")
        print(f"  Type: {msg_type.upper()}")
        print(f"  Content: {msg_content}")
        print("-" * 40)
    except json.JSONDecodeError:
        print(f"ID: {msg_id}, Session: {session_id}, Message: {message_json[:100]}...") # JSON 파싱 오류 시 원본 출력
        print("-" * 40)

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
store_histories = {} # SQLChatMessageHistory 객체를 저장
# 데이터베이스에서 대화내용 가져오는 함수
def get_chat_history(session_id):
  db_path = f"/content/drive/MyDrive/LangChain/{session_id}_chat.db"
  if session_id not in store_histories:
      print(f"{session_id} 에 대한 SQLChatMessageHistory 생성 및 연결")
      sql_chat_history = SQLChatMessageHistory(
            session_id=session_id,
            connection=f"sqlite:///{db_path}"
      )
      store_histories[session_id] = sql_chat_history
  return store_histories[session_id] # SQLChatMessageHistory 객체 반환

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 한국어 챗봇입니다."),
    MessagesPlaceholder(variable_name="history"), # 현재 대화 기록
    ("human", "{input}"),
])

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

# ConversationSummaryBufferMemory 사용
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=200,  # 요약의 기준이 되는 토큰 길이 설정
    return_messages=True,
)

<ipython-input-6-85111161>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


# ConversationChain 사용

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain
conversation = ConversationChain(
    llm = llm,
    memory=memory,
    prompt=prompt,
)

<ipython-input-9-3524113137>:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
conversation_chain_with_history = RunnableWithMessageHistory(
    conversation, # conversationChain
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history", #prompt의 {history}에 대화 기록 전달
)

In [ ]:
def chatbot(session_id):
  print("\n--- 대화형 챗봇 시작 ---")
  print("종료하려면 'exit' 또는 'quit'을 입력하세요.")
  print("챗봇: 안녕하세요. 무엇을 도와드릴까요?")
  while True:
    user_input = input("입력: ")
    if user_input.lower() in ["exit", "quit"]:
        print("챗봇을 종료합니다.")
        break
    print("챗봇 (답변 생성 중)...")
    try:
      config = {"configurable": {"session_id": session_id}}
      inputs = {"input": user_input}
      response = conversation_chain_with_history.invoke(inputs, config)['response']
      print(response)
      print("\n--- Summarizer Memory의 요약이 업데이트되었습니다. ---")
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

In [ ]:
chatbot("session1_r")


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
입력: 서울에 대해 간략하게 설명해줘
챗봇 (답변 생성 중)...
session1_r 에 대한 SQLChatMessageHistory 생성 및 연결
서울은 대한민국의 수도이자 가장 큰 도시로, 역사와 현대가 공존하는 매력적인 곳입니다.  한강을 중심으로 발전해왔으며, 고궁과 같은 유서 깊은 문화유적과 최첨단 건물, 번화한 상업지구가 조화롭게 어우러져 있습니다.  세계적인 K-팝과 K-드라마의 중심지이기도 하며, 다양한 음식, 쇼핑, 엔터테인먼트를 즐길 수 있습니다.  짧게 말하자면, 역동적이고 활기찬 도시이면서 동시에 아름다운 자연과 풍부한 역사를 간직한 매력적인 도시입니다.

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: 내 이름은 테디야
챗봇 (답변 생성 중)...
반갑습니다, 테디님!  무엇을 도와드릴까요?

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: quit
챗봇을 종료합니다.


In [ ]:
#메모리 확인
conversation.memory.load_memory_variables({})['history']

[SystemMessage(content="The AI describes Seoul as South Korea's capital and largest city, a fascinating blend of history and modernity centered around the Han River.  It features a harmonious mix of historical cultural assets like palaces and modern architecture and bustling commercial districts.  It's also a global center for K-pop and entertainment, offering diverse food and shopping experiences.  In short, it's described as a dynamic and vibrant city well worth visiting.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내 이름은 테디야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='반갑습니다, 테디님!  무엇을 도와드릴까요? 😊', additional_kwargs={}, response_metadata={})]

In [ ]:
#데이터베이스 확인
get_history_from_db('session1_r')

데이터베이스 '/content/drive/MyDrive/LangChain/session1_r_chat.db'에 성공적으로 연결되었습니다.

대화 기록
ID: 1, Session: session1_r
  Type: HUMAN
  Content: 서울에 대해 간략하게 설명해줘
----------------------------------------
ID: 2, Session: session1_r
  Type: AI
  Content: 서울은 대한민국의 수도이자 가장 큰 도시로, 역사와 현대가 공존하는 매력적인 곳입니다.  한강을 중심으로 발전해왔으며, 고궁과 같은 유서 깊은 문화유적과 최첨단 건물, 번화한 상업지구가 조화롭게 어우러져 있습니다.  세계적인 K-팝과 K-드라마의 중심지이기도 하며, 다양한 음식, 쇼핑, 엔터테인먼트를 즐길 수 있습니다.  짧게 말하자면, 역동적이고 활기찬 도시이면서 동시에 아름다운 자연과 풍부한 역사를 간직한 매력적인 도시입니다.
----------------------------------------
ID: 3, Session: session1_r
  Type: HUMAN
  Content: 내 이름은 테디야
----------------------------------------
ID: 4, Session: session1_r
  Type: AI
  Content: 반갑습니다, 테디님!  무엇을 도와드릴까요?
----------------------------------------


In [ ]:
#세션2
chatbot("session2_r")


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
입력: 내 이름을 기억해?
챗봇 (답변 생성 중)...
session2_r 에 대한 SQLChatMessageHistory 생성 및 연결
네, 테디님!  이름을 기억하고 있습니다.  무엇이든 편하게 말씀해주세요.

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: 버스의 종류를 알려줄래?
챗봇 (답변 생성 중)...
버스의 종류는 여러 가지 기준으로 나눌 수 있습니다.  크게는 크기, 운행 구간, 운행 방식 등으로 분류할 수 있죠.  한국에서 흔히 볼 수 있는 버스 종류를 예시로 설명해 드릴게요.

**크기:**

* **소형버스:**  승객 수용 인원이 적은 버스로, 주로 마을버스나 특정 지역을 운행하는 셔틀버스 등에 사용됩니다.
* **중형버스:**  소형버스보다 크고 대형버스보다 작은 버스로, 시내버스나 광역버스 등에 다양하게 사용됩니다.
* **대형버스:**  승객 수용 인원이 가장 많은 버스로, 시외버스, 고속버스, 관광버스 등에 사용됩니다.

**운행 구간:**

* **시내버스:**  도시 내에서 운행하는 버스로, 노선이 다양하고 정류장이 많습니다.
* **마을버스:**  특정 지역이나 마을을 중심으로 운행하는 버스로, 시내버스보다 운행 범위가 작습니다.
* **광역버스:**  시 또는 도를 넘나들며 운행하는 버스로, 시내버스보다 운행 거리가 깁니다.
* **시외버스:**  시 또는 도를 넘나들며 운행하는 버스로, 광역버스와 비슷하지만, 더 먼 거리를 운행합니다.
* **고속버스:**  고속도로를 이용하여 장거리를 빠르게 운행하는 버스입니다.

**운행 방식:**

* **일반버스:**  정해진 노선을 따라 정류장에 정차하며 운행하는 버스입니다.
* **급행버스:**  일반버스보다 정류장 수가 적어 더 빠르게 목적지에 도착하는 버스입니다.
* **간선버스:**  주요 도로를 따라 운행하며

In [ ]:
#데이터베이스 확인
get_history_from_db('session2_r')

데이터베이스 '/content/drive/MyDrive/LangChain/session2_r_chat.db'에 성공적으로 연결되었습니다.

대화 기록
ID: 1, Session: session2_r
  Type: HUMAN
  Content: 내 이름을 기억해?
----------------------------------------
ID: 2, Session: session2_r
  Type: AI
  Content: 네, 테디님!  이름을 기억하고 있습니다.  무엇이든 편하게 말씀해주세요.
----------------------------------------
ID: 3, Session: session2_r
  Type: HUMAN
  Content: 버스의 종류를 알려줄래?
----------------------------------------
ID: 4, Session: session2_r
  Type: AI
  Content: 버스의 종류는 여러 가지 기준으로 나눌 수 있습니다.  크게는 크기, 운행 구간, 운행 방식 등으로 분류할 수 있죠.  한국에서 흔히 볼 수 있는 버스 종류를 예시로 설명해 드릴게요.

**크기:**

* **소형버스:**  승객 수용 인원이 적은 버스로, 주로 마을버스나 특정 지역을 운행하는 셔틀버스 등에 사용됩니다.
* **중형버스:**  소형버스보다 크고 대형버스보다 작은 버스로, 시내버스나 광역버스 등에 다양하게 사용됩니다.
* **대형버스:**  승객 수용 인원이 가장 많은 버스로, 시외버스, 고속버스, 관광버스 등에 사용됩니다.

**운행 구간:**

* **시내버스:**  도시 내에서 운행하는 버스로, 노선이 다양하고 정류장이 많습니다.
* **마을버스:**  특정 지역이나 마을을 중심으로 운행하는 버스로, 시내버스보다 운행 범위가 작습니다.
* **광역버스:**  시 또는 도를 넘나들며 운행하는 버스로, 시내버스보다 운행 거리가 깁니다.
* **시외버스:**  시 또는 도를 넘나들며 운행하는 버스로, 광역버스와

#메모리 직접 사용

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
chain_with_history = RunnableWithMessageHistory(
    prompt|llm,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history",
    verbose=True,
)

In [ ]:
def chatbot(session_id):
  print("\n--- 대화형 챗봇 시작 ---")
  print("종료하려면 'exit' 또는 'quit'을 입력하세요.")
  print("챗봇: 안녕하세요. 무엇을 도와드릴까요?")
  while True:
    user_input = input("입력: ")
    if user_input.lower() in ["exit", "quit"]:
        print("챗봇을 종료합니다.")
        break
    # config 설정. 세션 ID 기준으로 대화 기록
    print("챗봇 (답변 생성 중)...")
    try:
      # ConversationSummaryBufferMemory에서 요약된 내용 로드
      history = memory.load_memory_variables({})['history']
      print("\n--- history에 저장된 내용 ---")
      print(history)
      print("---history 내용 출력 끝---\n")
      config = {"configurable": {"session_id": session_id}}
      inputs = {"input": user_input, "history":history}
      response = chain_with_history.invoke(inputs, config).content
      print(response)
      # 사용자 입력과 AI의 답변을 ConversationSummaryMemory에 전달하여 요약 업데이트
      memory.save_context(
          {"input": user_input}, # 사용자 입력
          {"output": response} # AI의 답변
      )
      print("\n--- Summarizer Memory의 요약이 업데이트되었습니다. ---")
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

In [ ]:
#세션1. 대화는 이어지는데 직접적으로 물어보면 기억을 못함
chatbot('session1_r')


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
입력: 이번엔 지형에 대해 설명해줄래?
챗봇 (답변 생성 중)...

--- history에 저장된 내용 ---
[SystemMessage(content="The human asks the AI for a brief explanation of Seoul. The AI responds that Seoul is South Korea's capital and largest city, a fascinating place where history and modernity coexist, developed around the Han River, blending historical sites with cutting-edge buildings and bustling commercial districts. It's also the center of global K-pop and K-dramas, offering diverse food, shopping, and entertainment; in short, a dynamic and vibrant city with beautiful nature and a rich history.  The conversation then shifts to the human introducing themselves as Teddy, and the AI acknowledging and remembering the name.  Finally, the human asks about different types of buses, and the AI provides a detailed explanation categorizing them by size (small, medium, large), operating routes (city, village, metropolitan, intercity, express), and opera

In [ ]:
#새로운 세션. 새로운 대화 시작
chatbot('session3_r')


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
입력: 기후에 대해 설명해줄래?
챗봇 (답변 생성 중)...

--- history에 저장된 내용 ---
[SystemMessage(content="The human asks the AI for a brief explanation of Seoul. The AI responds that Seoul is South Korea's capital and largest city, a fascinating place where history and modernity coexist, developed around the Han River, blending historical sites with cutting-edge buildings and bustling commercial districts. It's also the center of global K-pop and K-dramas, offering diverse food, shopping, and entertainment; in short, a dynamic and vibrant city with beautiful nature and rich history. The conversation then shifts to the human introducing themselves as Teddy, and the AI acknowledging and remembering the name.  The human then asks about different types of buses, and the AI provides a detailed explanation categorizing them by size (small, medium, large), operating routes (city, village, metropolitan, intercity, express), and operating method

In [ ]:
#메모리 확인
memory.load_memory_variables({})["history"]

[SystemMessage(content="The human, Teddy, initially asks the AI for a brief explanation of Seoul, which the AI provides, detailing its history, modernity, geography centered around the Han River, and its status as a global hub for K-pop and K-dramas.  Teddy then asks about different types of buses, receiving a detailed categorization.  After thanking the AI (고마워) and receiving a polite response (천만에요!), Teddy asks about Seoul's geography in more detail, receiving a description of its mountainous north, the Han River, and the hills and plains in between. The AI explains how this combination creates a unique landscape.  The conversation then shifts. Teddy asks the AI if it remembers their previous conversation; the AI replies that it doesn't remember past conversations due to its short-term memory and offers to start a new conversation.  Teddy then asks the AI to explain climate, and the AI provides a detailed explanation of what climate is, the factors that determine it (latitude, altit

In [ ]:
#데이터베이스 확인
get_history_from_db('session3_r')

데이터베이스 '/content/drive/MyDrive/LangChain/session3_r_chat.db'에 성공적으로 연결되었습니다.

대화 기록
ID: 1, Session: session3_r
  Type: HUMAN
  Content: 기후에 대해 설명해줄래?
----------------------------------------
ID: 2, Session: session3_r
  Type: AI
  Content: 안녕하세요! 기후에 대해 설명해 드릴게요.  쉽고 친절하게 설명해 드리겠습니다.

**기후란 무엇일까요?**

기후는 어떤 지역의 **장기적인 대기 상태**를 말합니다.  단순히 오늘, 내일의 날씨가 아니라, 그 지역의 평균적인 온도, 강수량, 습도, 바람, 일조량 등을 수십 년, 수백 년 동안 관측하여 얻은 통계적인 자료를 바탕으로 정의됩니다.  보통 30년 이상의 기간을 가지고 평균적인 기상 상태를 기후라고 합니다.  날씨는 매일매일 변하지만, 기후는 상대적으로 안정적이라고 볼 수 있죠.

**기후를 결정하는 요인은 무엇일까요?**

기후는 여러 요인의 복합적인 결과입니다.  주요 요인으로는 다음과 같은 것들이 있습니다.

* **위도:** 위도가 높을수록 태양 에너지를 덜 받아 기온이 낮아집니다.  극지방은 추운 기후, 적도 지방은 더운 기후를 갖는 이유입니다.
* **고도:** 고도가 높을수록 기온이 낮아집니다.  산 정상은 산 기슭보다 훨씬 춥습니다.
* **해양의 영향:** 바다의 영향을 많이 받는 지역은 기온의 변화가 적고 습도가 높은 해양성 기후를 갖습니다.  반대로 내륙 지역은 기온의 변화가 크고 건조한 대륙성 기후를 갖습니다.
* **대륙의 영향:** 대륙의 크기와 지형도 기후에 영향을 미칩니다.  큰 대륙은 내륙 지역의 기온 변화를 크게 만듭니다.
* **해류:** 따뜻한 해류는 주변 지역의 기온을 높이고, 차가운 해류는 주변 지역의 기온을 낮춥니다.  멕시코 만류처럼 따뜻한 해류는 서유럽의 기후를 온화하게 만드는 중요